In [76]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso, LassoCV, LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import (GridSearchCV, cross_val_score,
                                     train_test_split)
from sklearn.preprocessing import StandardScaler

In [77]:
data_path = '../mlcourse.ai_Dataset/'

data = pd.read_csv(data_path + 'winequality-white.csv')
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [78]:
y = data.pop('quality')

X_train, X_holdout, y_train, y_holdout = train_test_split(data, y, test_size=0.3, random_state=17)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_holdout_scaled = scaler.transform(X_holdout)

In [79]:
linreg = LinearRegression()
linreg.fit(X_train_scaled, y_train)

LinearRegression()

In [80]:
print("Mean squared error (train): %.3f" % mean_squared_error(y_train, linreg.predict(X_train_scaled)))
print("Mean squared error (test): %.3f" % mean_squared_error(y_holdout, linreg.predict(X_holdout_scaled)))

Mean squared error (train): 0.558
Mean squared error (test): 0.584


In [81]:
linreg_coef = pd.DataFrame(np.abs(linreg.coef_), columns=['Coefficient'])
linreg_coef.sort_values(by='Coefficient', ascending=False)

,Coefficient
7,0.665720
3,0.538164
1,0.192260
8,0.150036
10,0.129533
0,0.097822
9,0.062053
5,0.042180
6,0.014304
4,0.008127


Density is the most influential for linear regression model

In [82]:
lasso1 = Lasso(alpha=0.01, random_state=17)
lasso1.fit(X_train_scaled, y_train)

Lasso(alpha=0.01, random_state=17)

In [83]:
lasso1_coef = pd.DataFrame(np.abs(lasso1.coef_), columns=['Coefficient'])
lasso1_coef.sort_values(by='Coefficient', ascending=False)

,Coefficient
10,0.322425
3,0.256363
7,0.235492
1,0.188479
8,0.067277
5,0.043088
9,0.029722
4,0.002747
0,0.000000
2,0.000000


Alcoghol was the most influential feature using Linear regression with lasso regularization

In [84]:
alphas = np.logspace(-6, 2, 200)
lasso_cv = LassoCV(alphas=alphas, random_state=17)
lasso_cv.fit(X_train_scaled, y_train)

LassoCV(alphas=array([1.00000000e-06, 1.09698580e-06, 1.20337784e-06, 1.32008840e-06,
       1.44811823e-06, 1.58856513e-06, 1.74263339e-06, 1.91164408e-06,
       2.09704640e-06, 2.30043012e-06, 2.52353917e-06, 2.76828663e-06,
       3.03677112e-06, 3.33129479e-06, 3.65438307e-06, 4.00880633e-06,
       4.39760361e-06, 4.82410870e-06, 5.29197874e-06, 5.80522552e-06,
       6.36824994e-06, 6.98587975e-0...
       1.18953407e+01, 1.30490198e+01, 1.43145894e+01, 1.57029012e+01,
       1.72258597e+01, 1.88965234e+01, 2.07292178e+01, 2.27396575e+01,
       2.49450814e+01, 2.73644000e+01, 3.00183581e+01, 3.29297126e+01,
       3.61234270e+01, 3.96268864e+01, 4.34701316e+01, 4.76861170e+01,
       5.23109931e+01, 5.73844165e+01, 6.29498899e+01, 6.90551352e+01,
       7.57525026e+01, 8.30994195e+01, 9.11588830e+01, 1.00000000e+02]),
        random_state=17)

In [85]:
lasso_cv.alpha_

0.0002833096101839324

In [86]:
lasso_cv_coef = pd.DataFrame(np.abs(lasso_cv.coef_), columns=['Coefficient'])
lasso_cv_coef.sort_values(by='Coefficient', ascending=False)

,Coefficient
7,0.648161
3,0.526883
1,0.192049
8,0.146549
10,0.137115
0,0.093295
9,0.060939
5,0.042698
6,0.012969
4,0.006933


Density was the most influential feature for Linear regression with Lasso regularization even after performing parameter search

In [87]:
print("Mean squared error (train): %.3f" % mean_squared_error(y_train, lasso_cv.predict(X_train_scaled)))
print("Mean squared error (test): %.3f" % mean_squared_error(y_holdout, lasso_cv.predict(X_holdout_scaled)))

Mean squared error (train): 0.558
Mean squared error (test): 0.583


In [88]:
forest = RandomForestRegressor(random_state=17, n_jobs=-1)
forest.fit(X_train_scaled, y_train)

RandomForestRegressor(n_jobs=-1, random_state=17)

In [89]:
print("Mean squared error (train): %.3f" % mean_squared_error(y_train, forest.predict(X_train_scaled)))
print("Mean squared error (cv): %.3f" % np.mean(np.abs(cross_val_score(forest, X_train_scaled, y_train, scoring="neg_mean_squared_error"))))
print("Mean squared error (test): %.3f" % mean_squared_error(y_holdout, forest.predict(X_holdout_scaled)))

Mean squared error (train): 0.053
Mean squared error (cv): 0.414
Mean squared error (test): 0.372


In [91]:
forest_params = {'max_depth': list(range(10, 25)),
                 'min_samples_leaf': list(range(1, 8)),
                 'max_features': list(range(6,12))}

locally_best_forest = GridSearchCV(RandomForestRegressor(n_jobs=-1, random_state=17), forest_params, scoring="neg_mean_squared_error", n_jobs=-1, cv=5, verbose=1)
locally_best_forest.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 630 candidates, totalling 3150 fits


KeyboardInterrupt: 

In [ ]:
print("Mean squared error (cv): %.3f" % mean_squared_error(y_holdout, locally_best_forest.predict(X_train_scaled)))
print("Mean squared error (test): %.3f" % mean_squared_error(y_train, locally_best_forest.predict(X_holdout_scaled)))

In [ ]:
rf_importance = pd.DataFrame(locally_best_forest.best_estimator_)
rf_importance.sort_values(by='importance', ascending=False)